In [ ]:
# Installing libraries
!pip install 'dask[complete]'

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os
import time
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point
from matplotlib.ticker import MultipleLocator
import dask.array as da
import dask.bag as db
import dask.dataframe as dd


In [ ]:
# Mount the Google Drive at /content/drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# The data was gathered from https://firms.modaps.eosdis.nasa.gov/download/create.php

In [ ]:
# Load the NASA FIRMS data

# J1 VIIRS C2
j1_viirs_path = '/content/drive/MyDrive/0.Data science final project/NASA Fire data/J1_VIIRS_C2/fire_archive_J1V-C2_587189.csv'
j1_viirs_archive_path = '/content/drive/MyDrive/0.Data science final project/NASA Fire data/J1_VIIRS_C2/fire_nrt_J1V-C2_587189.csv'
J1_data = pd.read_csv(j1_viirs_path)
J1_archive_data = pd.read_csv(j1_viirs_archive_path)

# J2 VIIRS C2
j2_viirs_path = '/content/drive/MyDrive/0.Data science final project/NASA Fire data/J2_VIIRS_C2/fire_nrt_J2V-C2_587190.csv'
J2_data = pd.read_csv(j2_viirs_path)

# MODIS data
modis_path = '/content/drive/MyDrive/0.Data science final project/NASA Fire data/MODIS_C6.1/fire_nrt_M-C61_587187.csv'
modis_archive_path = '/content/drive/MyDrive/0.Data science final project/NASA Fire data/MODIS_C6.1/fire_archive_M-C61_587187.csv'
modis_data = pd.read_csv(modis_path)
modis_archive_data = pd.read_csv(modis_archive_path)

#SUOMI data
suomi_path = '/content/drive/MyDrive/0.Data science final project/NASA Fire data/SUOMI_VIIRS_C2/fire_nrt_SV-C2_587191.csv'
suomi_archive_path = '/content/drive/MyDrive/0.Data science final project/NASA Fire data/SUOMI_VIIRS_C2/fire_archive_SV-C2_587191.csv'
suomi_data = pd.read_csv(suomi_path)
suomi_archive_data = pd.read_csv(suomi_archive_path)


In [ ]:
# Explanation of columns
# Brightness = VIIRS I-4 channel brightness temperature of the fire pixel measured in Kelvin.
# scan = along scan pixel size
# track = along track pixel size
# "Scan" and "track" values provide information about the actual spatial resolution of the detected fire pixel, which can vary depending on its location within the sensor's field of view.
# confidence = l low, n nominal, h high
# version = collection and soure version
# bright_t31 = I-5 Channel brightness temperature of the fire pixel measured in Kelvin.
# FRP - fire radiative power
# acq date/time, date and time of the acquisition. Time in UTC
# type = 0 presumed vegetation fire, 1 active volcano, 2 other static land source, 3 offshore detection - only available for standard quality data
# Further information can be found here https://firms.modaps.eosdis.nasa.gov/descriptions/FIRMS_VIIRS_Firehotspots.html

# difference between Brightness and bright t_31 - brightness is i4 which is in the mid infrared spectral region
# i5 is a slightly different wavelength, and provides complementary information to i4


In [ ]:
J1_data.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,60.39159,22.76142,307.01,0.53,0.50,2024-03-10,0,N20,VIIRS,n,2,264.55,1.66,N,0
1,52.55890,39.57714,329.25,0.53,0.42,2024-03-10,1,N20,VIIRS,n,2,257.88,6.99,N,2
2,52.55527,39.57491,330.11,0.53,0.42,2024-03-10,1,N20,VIIRS,n,2,260.44,6.99,N,2
3,54.16821,37.71584,322.69,0.44,0.38,2024-03-10,1,N20,VIIRS,n,2,260.96,3.08,N,2
4,53.26996,24.44065,301.30,0.39,0.44,2024-03-10,2,N20,VIIRS,n,2,269.27,0.97,N,2


In [ ]:
J1_archive_data.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,9.22153,25.55140,304.64,0.46,0.39,2024-12-01,0,N20,VIIRS,n,2.0NRT,287.24,0.64,N
1,10.47126,21.06918,300.92,0.40,0.37,2024-12-01,0,N20,VIIRS,n,2.0NRT,288.27,0.63,N
2,11.50480,13.61005,296.90,0.37,0.58,2024-12-01,0,N20,VIIRS,n,2.0NRT,283.22,0.37,N
3,9.69235,25.85875,300.88,0.48,0.40,2024-12-01,0,N20,VIIRS,n,2.0NRT,287.82,0.75,N
4,9.71555,25.69700,327.73,0.47,0.39,2024-12-01,0,N20,VIIRS,n,2.0NRT,289.12,2.45,N


In [ ]:
J2_data.tail()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
21813186,4.58452,9.89171,297.61,0.68,0.74,2025-03-09,2358,N21,VIIRS,n,2.0NRT,285.21,2.61,N
21813187,4.58532,9.88581,296.36,0.68,0.74,2025-03-09,2358,N21,VIIRS,n,2.0NRT,286.02,2.28,N
21813188,4.28499,12.12092,305.29,0.47,0.64,2025-03-09,2358,N21,VIIRS,n,2.0NRT,287.89,1.45,N
21813189,4.13608,13.75399,302.39,0.36,0.58,2025-03-09,2358,N21,VIIRS,n,2.0NRT,288.03,0.78,N
21813190,4.21056,13.50780,302.57,0.38,0.59,2025-03-09,2358,N21,VIIRS,n,2.0NRT,289.14,0.64,N


In [ ]:
modis_data.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,35.53210,44.34115,300.57,1.93,1.36,2024-12-01,17,Aqua,MODIS,30,6.1NRT,274.40,23.37,N
1,35.68364,43.87657,312.58,1.79,1.31,2024-12-01,17,Aqua,MODIS,85,6.1NRT,273.94,39.19,N
2,33.72323,46.29848,302.86,2.87,1.62,2024-12-01,17,Aqua,MODIS,52,6.1NRT,272.96,48.31,N
3,34.89808,43.81138,318.45,1.85,1.33,2024-12-01,17,Aqua,MODIS,97,6.1NRT,277.52,52.14,N
4,34.90350,43.80552,316.31,1.85,1.33,2024-12-01,17,Aqua,MODIS,93,6.1NRT,276.10,47.52,N


In [ ]:
modis_archive_data.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,-11.6272,-45.1273,309.4,2.4,1.5,2024-03-10,32,Terra,MODIS,77,61.03,293.1,28.3,N,0
1,-11.6405,-45.1259,310.5,2.4,1.5,2024-03-10,32,Terra,MODIS,80,61.03,293.5,30.0,N,0
2,-12.9574,-45.1309,307.4,2.5,1.5,2024-03-10,32,Terra,MODIS,68,61.03,292.0,26.3,N,0
3,-10.2513,-37.8315,318.7,1.0,1.0,2024-03-10,32,Terra,MODIS,97,61.03,295.4,14.4,N,0
4,-10.0524,-38.1128,306.0,1.0,1.0,2024-03-10,32,Terra,MODIS,59,61.03,295.2,4.4,N,0


In [ ]:
suomi_data.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,-3.86309,10.94714,315.51,0.41,0.60,2025-01-01,0,N,VIIRS,n,2.0NRT,285.53,1.17,N
1,-4.92408,11.37806,341.24,0.37,0.58,2025-01-01,0,N,VIIRS,n,2.0NRT,284.04,5.36,N
2,-4.86461,12.02828,301.02,0.33,0.55,2025-01-01,0,N,VIIRS,n,2.0NRT,285.07,3.26,N
3,-4.92875,12.08102,297.53,0.33,0.55,2025-01-01,0,N,VIIRS,n,2.0NRT,282.82,0.85,N
4,-4.86307,12.01922,301.00,0.33,0.55,2025-01-01,0,N,VIIRS,n,2.0NRT,284.67,3.96,N


In [ ]:
suomi_archive_data.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,68.41540,83.62521,367.00,0.59,0.70,2024-03-10,44,N,VIIRS,h,2,247.23,11.63,N,2
1,67.59406,83.25334,327.22,0.67,0.73,2024-03-10,44,N,VIIRS,n,2,239.85,3.71,N,2
2,67.59660,83.25000,344.16,0.67,0.73,2024-03-10,44,N,VIIRS,n,2,241.13,6.39,N,2
3,67.51987,83.21684,324.17,0.67,0.74,2024-03-10,44,N,VIIRS,n,2,240.71,3.32,N,2
4,67.28148,83.04497,327.03,0.70,0.75,2024-03-10,44,N,VIIRS,n,2,241.65,5.99,N,2


In [ ]:
# replacing values in the satellite column so rows can be identifiable by source once they are joined together
suomi_data ['satellite'] = suomi_data['satellite'].replace(['N'], 'SUOMI')
suomi_archive_data ['satellite'] = suomi_archive_data['satellite'].replace(['N'], 'SUOMI')
J1_data ['satellite'] = J1_data['satellite'].replace(['N20'], 'NOAA-20')
J1_archive_data ['satellite'] = J1_archive_data['satellite'].replace(['N20'], 'NOAA-20')
J2_data ['satellite'] = J2_data['satellite'].replace(['N21'], 'NOAA-21')

In [ ]:
# concatenating data by satellite source as the archive file covers the period from March 2024 to Dec 2024 (Suomi, Modis and J1 only. J2 and Landsat files cover the entire period)
suomi = [suomi_archive_data, suomi_data]
suomi_data = pd.concat(suomi)
modis = [modis_archive_data, modis_data]
modis_data = pd.concat(modis)
j1 = [J1_archive_data, J1_data]
j1_data = pd.concat(j1)


In [ ]:
# concatenating all files together
firms = [suomi_data, modis_data, j1_data, J2_data]
all_firms_data = pd.concat(firms)


In [ ]:
# find null values in the dataframe
all_firms_data.isnull().sum()

,0
latitude,0
longitude,0
brightness,0
scan,0
track,0
acq_date,0
acq_time,0
satellite,0
instrument,0
confidence,0


In [ ]:
all_firms_data['type'] = all_firms_data['type'].astype(str)

# change null values in the type column to "no type available" as this is not available in the data
all_firms_data['type'] = all_firms_data['type'].fillna('No type available')
# replace numerical values with description in the Type column
all_firms_data['type'] = all_firms_data['type'].replace(['0.0'], 'Presumed Vegetation Fire')
all_firms_data['type'] = all_firms_data['type'].replace(['1.0'], 'Active Volcano')
all_firms_data['type'] = all_firms_data['type'].replace(['2.0'], 'Other Static Land Source')
all_firms_data['type'] = all_firms_data['type'].replace(['3.0'], 'Offshore Detection')

In [ ]:
# confidence column contains both numerical values, and expected values of l (low), n (nominal), h (high)
# find out the numerical values and the max-min range
all_firms_data['confidence'].unique()

array(['h', 'n', 'l', 77, 80, 68, 97, 59, 61, 55, 81, 100, 66, 72, 71, 75,
       73, 62, 87, 69, 25, 53, 45, 82, 54, 57, 41, 70, 98, 83, 36, 94, 22,
       47, 64, 50, 56, 48, 67, 89, 42, 49, 24, 38, 0, 60, 84, 30, 26, 43,
       51, 79, 63, 65, 96, 78, 74, 91, 5, 35, 58, 90, 27, 92, 88, 86, 44,
       10, 99, 40, 93, 95, 8, 17, 46, 52, 16, 85, 6, 12, 20, 31, 14, 33,
       34, 18, 21, 39, 23, 76, 32, 37, 29, 19, 15, 28, 11, 9, 13, 4, 7, 3,
       2, 1], dtype=object)

In [ ]:
# replace numerical values with expected column value of l/n/h
# replace 1 to 33 with l
# replace 34 to 66 with n
# replace 67 to 100 with h
all_firms_data['confidence'] = all_firms_data['confidence'].replace([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33], 'l')
all_firms_data['confidence'] = all_firms_data['confidence'].replace([34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66], 'n')
all_firms_data['confidence'] = all_firms_data['confidence'].replace([67,68,69,70, 71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100], 'h')

In [ ]:
# Drop version column as this does contain data as expected
all_firms_data = all_firms_data.drop('version', axis=1)

In [ ]:
# get stats about the dataset
all_firms_data.describe()

,latitude,longitude,brightness,scan,track,acq_time,bright_t31,frp
count,7.025128e+07,7.025128e+07,7.025128e+07,7.025128e+07,7.025128e+07,7.025128e+07,7.025128e+07,7.025128e+07
mean,5.792820e+00,1.612951e+01,3.335839e+02,5.360065e-01,5.306205e-01,1.201179e+03,2.993745e+02,1.258234e+01
std,2.385047e+01,6.607144e+01,1.782269e+01,3.552850e-01,2.215292e-01,5.494236e+02,1.080354e+01,5.329954e+02
min,-8.861739e+01,-1.799253e+02,0.000000e+00,3.200000e-01,3.600000e-01,0.000000e+00,0.000000e+00,-3.550000e+00
25%,-1.265326e+01,-4.385651e+01,3.237000e+02,4.000000e-01,3.800000e-01,8.450000e+02,2.924100e+02,2.880000e+00
50%,9.013800e-01,2.289832e+01,3.361100e+02,4.500000e-01,4.600000e-01,1.157000e+03,3.001000e+02,5.730000e+00
75%,1.727778e+01,3.962943e+01,3.443900e+02,5.300000e-01,5.900000e-01,1.637000e+03,3.064300e+02,1.132000e+01
max,8.562080e+01,1.799608e+02,5.100000e+02,4.820000e+00,2.000000e+00,2.359000e+03,4.233400e+02,4.456712e+06


In [ ]:
all_firms_data.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,bright_t31,frp,daynight,type
0,68.41540,83.62521,367.00,0.59,0.70,2024-03-10,44,SUOMI,VIIRS,h,247.23,11.63,N,Other Static Land Source
1,67.59406,83.25334,327.22,0.67,0.73,2024-03-10,44,SUOMI,VIIRS,n,239.85,3.71,N,Other Static Land Source
2,67.59660,83.25000,344.16,0.67,0.73,2024-03-10,44,SUOMI,VIIRS,n,241.13,6.39,N,Other Static Land Source
3,67.51987,83.21684,324.17,0.67,0.74,2024-03-10,44,SUOMI,VIIRS,n,240.71,3.32,N,Other Static Land Source
4,67.28148,83.04497,327.03,0.70,0.75,2024-03-10,44,SUOMI,VIIRS,n,241.65,5.99,N,Other Static Land Source


In [ ]:
# filter the dataset for lat and long
# target lat 32.0 to 49.0
# target long -125 to -114
firms_data_target = all_firms_data[(all_firms_data['latitude'] >= 32.0) & (all_firms_data['latitude'] <= 49.0) & (all_firms_data['longitude'] >= -125.0) & (all_firms_data['longitude'] <= -114.0)]

In [ ]:
# Feature engineering on the fire metrics
# create ratio between frp and brightness
frp_brightness_ratio = firms_data_target['frp'] / firms_data_target['brightness']
firms_data_target['frp_brightness_ratio'] = frp_brightness_ratio

<ipython-input-28-3d9cd2667065>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  firms_data_target['frp_brightness_ratio'] = frp_brightness_ratio


In [ ]:
# extract day of the week from acq_date column
firms_data_target['acq_date'] = pd.to_datetime(firms_data_target['acq_date'])
firms_data_target['day_of_week'] = firms_data_target['acq_date'].dt.day_name()

<ipython-input-29-931006c049bf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  firms_data_target['acq_date'] = pd.to_datetime(firms_data_target['acq_date'])
<ipython-input-29-931006c049bf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  firms_data_target['day_of_week'] = firms_data_target['acq_date'].dt.day_name()


In [ ]:
firms_data_target['type'] = firms_data_target['type'].fillna('No type available')

<ipython-input-30-39a2fc4519df>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  firms_data_target['type'] = firms_data_target['type'].fillna('No type available')


In [ ]:
fire_data_target = firms_data_target.copy()

In [ ]:
fire_data_target.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,bright_t31,frp,daynight,type,frp_brightness_ratio,day_of_week
11930,36.28835,-120.25271,298.26,0.42,0.45,2024-03-10,925,SUOMI,VIIRS,n,269.44,0.80,N,Presumed Vegetation Fire,0.002682,Sunday
11931,36.27491,-120.24533,304.08,0.42,0.45,2024-03-10,925,SUOMI,VIIRS,n,270.10,0.89,N,Presumed Vegetation Fire,0.002927,Sunday
11932,36.27073,-120.24596,301.97,0.42,0.45,2024-03-10,925,SUOMI,VIIRS,n,268.92,0.89,N,Presumed Vegetation Fire,0.002947,Sunday
11945,34.62195,-117.10254,310.34,0.43,0.38,2024-03-10,925,SUOMI,VIIRS,n,280.16,1.33,N,Presumed Vegetation Fire,0.004286,Sunday
11946,34.60886,-117.33659,308.74,0.44,0.39,2024-03-10,925,SUOMI,VIIRS,n,280.24,1.39,N,Presumed Vegetation Fire,0.004502,Sunday


In [ ]:
fire_data_target.info()

<class 'pandas.core.frame.DataFrame'>
Index: 588502 entries, 11930 to 21809969
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   latitude              588502 non-null  float64       
 1   longitude             588502 non-null  float64       
 2   brightness            588502 non-null  float64       
 3   scan                  588502 non-null  float64       
 4   track                 588502 non-null  float64       
 5   acq_date              588502 non-null  datetime64[ns]
 6   acq_time              588502 non-null  int64         
 7   satellite             588502 non-null  object        
 8   instrument            588502 non-null  object        
 9   confidence            588502 non-null  object        
 10  bright_t31            588502 non-null  float64       
 11  frp                   588502 non-null  float64       
 12  daynight              588502 non-null  object        
 13

In [ ]:
fire_data_target.describe()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,bright_t31,frp,frp_brightness_ratio
count,588502.000000,588502.000000,588502.000000,588502.000000,588502.000000,588502,588502.000000,588502.000000,588502.000000,588502.000000
mean,42.441721,-118.660449,327.485722,0.552084,0.543657,2024-08-24 06:58:23.776028160,1393.519592,293.455336,22.972428,0.064244
min,32.001000,-124.931980,207.380000,0.320000,0.360000,2024-03-10 00:00:00,345.000000,233.420000,0.000000,0.000000
25%,40.157378,-120.794500,307.092500,0.400000,0.380000,2024-07-25 00:00:00,940.000000,284.900000,1.930000,0.006198
50%,44.142830,-118.910805,327.500000,0.450000,0.470000,2024-08-19 00:00:00,1032.000000,290.570000,4.900000,0.014862
75%,44.935620,-116.095840,344.180000,0.540000,0.610000,2024-09-10 00:00:00,2035.000000,300.050000,15.000000,0.043832
max,48.999750,-114.000000,509.500000,4.800000,2.000000,2025-03-09 00:00:00,2305.000000,400.100000,12423.700000,24.847400
std,4.020046,2.611640,22.716401,0.401031,0.235824,NaN,554.473125,14.133239,100.018190,0.240724


In [ ]:
# Save the file in Drive so above code doesn't have to be run every time, instead the file can be directly loaded from Drive
save_path = '/content/drive/MyDrive/0.Data science final project/NASA Fire data/fire_data_target.csv'
fire_data_target.to_csv(save_path, index=False)

In [ ]:
# Load the dataset
load_path = '/content/drive/MyDrive/0.Data science final project/NASA Fire data/fire_data_target.csv'
firms_data_target = pd.read_csv(load_path)